<a href="https://colab.research.google.com/github/kush1305/Satellite-Imagery-Based-Property-Valuation-/blob/main/data_fetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import math
import requests
import pandas as pd
from tqdm import tqdm

# -----------------------------
# CONFIG
# -----------------------------

DATA_PATH = "" ##Add traing data path file
IMAGE_DIR = "" ## Add the path where you want to save the images

ZOOM = 16

os.makedirs(IMAGE_DIR, exist_ok=True)

# -----------------------------
# LAT/LON → TILE CONVERSION
# -----------------------------

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int(
        (1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / 2.0 / math.pi * n)
    )
    return xtile, ytile

# -----------------------------
# FETCH IMAGE
# -----------------------------

def fetch_satellite_image(lat, lon, save_path):
    x, y = deg2num(lat, lon, ZOOM)
    url = (
        "https://services.arcgisonline.com/ArcGIS/rest/services/"
        f"World_Imagery/MapServer/tile/{ZOOM}/{y}/{x}"
    )

    response = requests.get(url, headers={"User-Agent": "Satellite-Downloader"})

    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
        return True
    else:
        print("❌ Failed:", lat, lon)
        return False

# -----------------------------
# MAIN
# -----------------------------

def main():
    df = pd.read_excel(DATA_PATH)

    for _, row in tqdm(df.iterrows(), total=len(df)):
        prop_id = row["id"]
        lat = row["lat"]
        lon = row["long"]

        save_path = os.path.join(IMAGE_DIR, f"{prop_id}.png")

        if not os.path.exists(save_path):
            fetch_satellite_image(lat, lon, save_path)

    print("✅ All images downloaded.")

if __name__ == "__main__":
    main()
